<a href="https://colab.research.google.com/github/Leos1999/GoldPricePredictor/blob/master/GoldPricePredictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras import optimizers
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import ModelCheckpoint,LearningRateScheduler
from tensorflow.keras.layers import Dense, LSTM, Dropout

In [9]:
data_URL = "https://raw.githubusercontent.com/Leos1999/GoldPricePredictor/master/goldrate.csv"
data = pd.read_csv(data_URL)
data.head()
data = data.iloc[:,1].values.reshape(-1,1)
print(data.shape)
scaler = MinMaxScaler(feature_range = (0, 1))

(607, 1)


In [10]:
def train_test_split_norm(data):
  data_norm = scaler.fit_transform(data)

  size = 0.85
  train_size = int(len(data_norm) * size)
  test_size = len(data_norm) - train_size
  train, test = data_norm[0:train_size, :], data_norm[train_size:len(data_norm), :]
  print("Number of entries (training set, test set): " + str((len(train), len(test))))
  return train,test

In [11]:
def create_dataset(dataset, window_size):
    data_X, data_Y = [], []
    for i in range(len(dataset) - window_size - 1):
        a = dataset[i:(i + window_size), 0]
        data_X.append(a)
        data_Y.append(dataset[i + window_size, 0])
    return(np.array(data_X), np.array(data_Y))

In [12]:
def window_maker(data,window_size):
  train,test = train_test_split_norm(data)
  train_X, train_Y = create_dataset(train, window_size)
  test_X, test_Y = create_dataset(test, window_size)
  # print("Original training data shape:")
  # print(train_X.shape)
  train_X = np.reshape(train_X, (train_X.shape[0], 1, train_X.shape[1]))
  test_X = np.reshape(test_X, (test_X.shape[0], 1, test_X.shape[1]))
  # print("New training data shape:")
  # print(train_X.shape)
  return train_X,train_Y,test_X,test_Y,train,test